# Baixando dados e documentos com o Jus.br 

Este notebook demonstra como autenticar e obter um token de acesso para o portal JUSBR usando a classe `JUSBR_Scraper`.


In [1]:
import juscraper as jus
import json

jusbr = jus.scraper("jusbr")

## Autenticação manual via gov.br

1. Acesse https://www.jus.br
2. Faça login usando o gov.br
3. Entre na página <https://portaldeservicos.pdpj.jus.br/consulta> (ou clique no botão 'Consultar processo', que aparece após o login)
4. Na nova página, abra a aba Network do navegador (F12 ou 'Inspecionar elemento')
5. Atualize a página (F5 ou ctrl+R ou no botão atualizar)
6. Nas requisições que vão aparecer, procure a requisição que tem nome 'token'. Clique nela.
7. Na tela ao lado, clique em 'Resposta'.
8. Selecione e copie o campo 'access_token' que aparece lá.


In [2]:
import os
from dotenv import load_dotenv
from getpass import getpass

load_dotenv()

access_token = os.environ.get("JUSBR_ACCESS_TOKEN")
if access_token is None:
    # input como senha oculta
    access_token = getpass("Digite o token JWT: ")


In [3]:

jusbr.auth(token=access_token)

True

## Baixando metadados dos processos

Baixando uma lista de casos:

In [7]:
processos = ['3005317-12.2025.8.06.0000']

resultados = jusbr.cpopg(processos)

resultados

,processo,numeroProcesso,idCodexTribunal,detalhes
0,30053171220258060000,3005317-12.2025.8.06.0000,None,"{'nivelSigilo': 0, 'siglaTribunal': 'TJCE', 'n..."


In [8]:
result_list = resultados['detalhes'].iloc[0]

print(json.dumps(result_list['tramitacaoAtual']['grau'], indent=2, ensure_ascii=False))

{
  "sigla": "G2",
  "nome": "2° Grau",
  "numero": 2
}


Se quiser visualizar o texto do caso:

## Baixando autos dos processos

In [9]:
df_docs = jusbr.download_documents(resultados)

print(df_docs.shape)
df_docs.head(3)

(1, 18)


,numero_processo,idCodex,sequencia,nome,tipo,dataHoraJuntada,nivelSigilo,hrefTexto,hrefBinario,texto,_raw_text_api,_raw_binary_api,arquivo,idOrigem,idDocumento,descricao,tipoDocumento,dataJuntada
0,3005317-12.2025.8.06.0000,25491324303,4,Decisão.html,"{'codigo': 404, 'nome': 'Decisão', 'idCodex': ...",2025-05-29T18:02:53.803,PUBLICO,/processos/3005317-12.2025.8.06.0000/documento...,/processos/3005317-12.2025.8.06.0000/documento...,ESTADO DO CEARÁ\n \n PODER JUDICIÁRIO\n \n T...,\nESTADO DO CEARÁ\n \n PODER JUDICIÁRIO\n \n ...,"b'<p><span style=""font-family: arial, helvetic...","{'tipo': 'text/html', 'quantidadePaginas': 0, ...",21009045,None,None,None,None


In [10]:
df_docs.head(3)

,numero_processo,idCodex,sequencia,nome,tipo,dataHoraJuntada,nivelSigilo,hrefTexto,hrefBinario,texto,_raw_text_api,_raw_binary_api,arquivo,idOrigem,idDocumento,descricao,tipoDocumento,dataJuntada
0,3005317-12.2025.8.06.0000,25491324303,4,Decisão.html,"{'codigo': 404, 'nome': 'Decisão', 'idCodex': ...",2025-05-29T18:02:53.803,PUBLICO,/processos/3005317-12.2025.8.06.0000/documento...,/processos/3005317-12.2025.8.06.0000/documento...,ESTADO DO CEARÁ\n \n PODER JUDICIÁRIO\n \n T...,\nESTADO DO CEARÁ\n \n PODER JUDICIÁRIO\n \n ...,"b'<p><span style=""font-family: arial, helvetic...","{'tipo': 'text/html', 'quantidadePaginas': 0, ...",21009045,None,None,None,None


In [8]:
print(df_docs['texto'][0][:200])

TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO
COMARCA de São Vicente
Foro de São Vicente
1ª Vara Criminal
Rua Jacob Emmerich, 1367, Sala 13, Parque Bitarú - CEP 11310-906, Fone: (13) 2102-6454, São Vicen


## Baixando uma lista grande de processos

Se a lista de processos é muito grande, pode ser uma boa prática baixar um processo por vez e salvar os resultados em um arquivo parquet, por exemplo.

No script abaixo, criamos uma pasta dentro da pasta atual, chamada jusbr/cpopg/. Ali dentro, para cada processo, criamos uma pasta com o número do processo e salvamos 2 arquivos: 1 com os metadados daquele processo e 1 com os textos dos documentos. A chave para juntar as duas bases é o número do processo.

In [9]:
import os
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed

processos = ['15006452620228260536']

resultados = jusbr.cpopg(processos)

In [10]:

def download_document(proc_no, jusbr, verbose=True):
    if verbose:
        print(f"Baixando documentos do processo {proc_no}\n")
    
    resultado = jusbr.cpopg([proc_no])
    row = resultado.iloc[0]
    processo = row['processo']
    row_df = pd.DataFrame([row])
    # cria a pasta
    process_path = os.path.join('jusbr/cpopg', processo)
    os.makedirs(process_path, exist_ok=True)
    
    # baixa documentos do processo
    if verbose:
        print(f"Baixando autos do processo {proc_no}\n")
    
    df_docs = jusbr.download_documents(row_df)
    #df_docs = pd.DataFrame()
    # salva documentos
    df_docs.to_parquet(os.path.join(process_path, 'documents.parquet'), index=False)
    # salva metadata
    row_df.to_parquet(os.path.join(process_path, 'metadata.parquet'), index=False)

for proc_no in processos:
    download_document(proc_no, jusbr)

Baixando documentos do processo 15006452620228260536

Baixando autos do processo 15006452620228260536

